In [125]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from multiprocessing import Pool
from tqdm import tqdm
import pickle

Loading in all data for inspection:
0. data volume
1. field consistancy
2. missing values anywhere at all
3. data types
4. basic visualisation for relevant cols
5. etc

In [19]:
data_path = 'data/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [27]:
for file in onlyfiles:
    if file[:3] == 'LOG':
        print('the inspected log file is: ' + file)
    else:
        print('the inspected non log file is :' + file)
    print('it has: ', len(pd.read_csv('data/' + file, sep = ';', error_bad_lines = False)), 'lines')

the inspected non log file is :TABLE_CONVERSION.csv
it has:  580404 lines
the inspected non log file is :.DS_Store
it has:  0 lines
the inspected non log file is :CLIENTS.csv
it has:  3125800 lines
the inspected log file is: LOG_WEB_201710.csv
it has:  3301363 lines
the inspected log file is: LOG_WEB_201711.csv
it has:  5055599 lines
the inspected non log file is :CMD.csv
it has:  7323227 lines
the inspected log file is: LOG_WEB_201712.csv
it has:  4535451 lines
the inspected log file is: LOG_WEB_201708.csv
it has:  2189118 lines
the inspected log file is: LOG_WEB_201709.csv
it has:  2874217 lines
the inspected log file is: LOG_WEB_201801.csv


/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


it has:  8450825 lines
the inspected log file is: LOG_WEB_201802.csv
it has:  6063362 lines
the inspected log file is: LOG_WEB_201803.csv
it has:  2739719 lines


these files are rather large containing many lines, we should use a subset to test the robustness of the classification

In [24]:
data_201708 = pd.read_csv('data/LOG_WEB_201708.csv', sep=';')

In [35]:
colns = sorted([col for col in list(data_201708.columns) if col[:3] != 'Unn'])

In [37]:
for file in onlyfiles:
    if file[:3] == 'LOG':
        colns_check = sorted([col for col in list(pd.read_csv('data/' + file, sep = ';').columns) if col[:3] != 'Unn'])
        print('the columns match the example columns: ', colns_check == colns)
        if colns_check != colns:
            print('the file with unmatched cols is: ', file)

the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True


/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True


So the 201709 file is not matched, let's take a look at what's going on

In [33]:
data_201709 = pd.read_csv('data/LOG_WEB_201709.csv', sep=';')

In [97]:
data_201709.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'VISITOR_ID', 'CONNECTED_SESSION',
       'ID_SESSION', 'SESSION_START_DATE', 'EVENT_DATE', 'PAGES',
       'GLOBAL_SOURCES', 'DEVICE_TYPE', 'DEVICE_MODEL', 'LOADINGS'],
      dtype='object')

In [54]:
set(data_201709.GLOBAL_SOURCES)

{'Accès Direct',
 'Affiliation et partenaires',
 'Email marketing',
 'Liens Sponsorisés',
 'Moteurs',
 'Notoriété',
 'Publicité',
 'Sites affluents',
 'Sources naturelles personnalisées',
 'Webmails'}

Let's take a look at the non log files

In [131]:
conver = pd.read_csv('data/TABLE_CONVERSION_new.csv', sep = ';')

In [132]:
conver.head()

,Unnamed: 0,VISITOR_ID,CLIENT_NUMBER
0,0,153926,2117916268519517296
1,1,153987,-3983308646720649306
2,2,153992,-2525192206264180159
3,3,153996,1543724696132514059
4,4,154002,-3753347333864614269


In [133]:
clients = pd.read_csv('data/CLIENTS_new.csv', sep = ';')

In [134]:
clients.head()

,Unnamed: 0,CLIENT_NUMBER,CREATION_DATE,GENDER,BIRTH_YEAR,ZIPCODE
0,0,-1182315409949716431,2002-12-05,Mme,1955.0,68440
1,1,7142641182482142162,2002-12-05,Mme,NaN,24100
2,2,-4786858050561650068,2002-12-05,Mme,1945.0,39240
3,3,-8463163259230597037,2002-12-05,Mme,NaN,42290
4,4,-8047395753631590665,2003-10-06,Mme,1934.0,13006


In [135]:
CMD = pd.read_csv('data/CMD_new.csv', sep = ';')

In [136]:
CMD.head()

,Unnamed: 0,CLIENT_NUMBER,ORDER_NUMBER,ORDER_DATE,ORDER_CHANNEL,PRE_TAX_AMOUNT,CVIC
0,0,7178742953965764755,2302390,2010-01-05,Telephone,84.28,True
1,1,-2373390172893352743,2302392,2010-01-17,Telephone,97.85,True
2,2,-240944575938990062,2302393,2010-01-05,Telephone,58.45,True
3,3,1720395480962300217,2302394,2010-01-05,Telephone,31.19,True
4,4,7161131950067982732,2302396,2010-01-14,Telephone,37.45,True


In [51]:
set(CMD.ORDER_CHANNEL)

{'Courier', 'Internet', 'Magasin', 'Non Renseigne', 'Telephone'}

In [53]:
len(CMD.CLIENT_NUMBER), len(set(CMD.CLIENT_NUMBER))

(7323227, 1604670)

as expected, over the years, one clients made multiple purchases

As informed the supervisor, the CVIC column is the column that identifies the users who use promo codes

We have a general look around the order channel and the CVIC column

In [61]:
CMD = CMD.drop(['Unnamed: 0'], axis = 1)

In [63]:
len(CMD[['Internet' == channel for channel in CMD.ORDER_CHANNEL]]), len(CMD)

(1648148, 7323227)

In [64]:
CMD[['Internet' == channel for channel in CMD.ORDER_CHANNEL]].CVIC

633861     False
634168     False
634413     False
635627     False
636492     False
637017     False
637785     False
637833     False
638038     False
638045     False
1341334    False
1341945    False
1344864    False
1346825    False
1349597    False
1350807    False
1354861    False
1355086    False
1355754    False
1357128    False
1362250    False
1362834    False
1364217    False
1364964    False
1366807    False
1369690    False
2410345     True
2410346     True
2410347    False
2410348     True
           ...  
6724543     True
6724544     True
6724545     True
6724546     True
6724547    False
6724548    False
6724549    False
6724550    False
6724551    False
6724552    False
6724553    False
6724554    False
6724555     True
6724556    False
6724557    False
6724558    False
6724559    False
6724560     True
6724561    False
6724562    False
6724563     True
6724564     True
6724565    False
6724566    False
6724567     True
6724568     True
6724569    False
6724570    Fal

In [94]:
CMD.ORDER_DATE[0][:7]

'2010-01'

In [160]:
def desired_group(row):
    if row['CVIC'] == True:
        if row['ORDER_CHANNEL'] == 'Internet':
            if row['ORDER_DATE'][:4] == '2017':
                if int(row['ORDER_DATE'][5:7]) >= 8:
                    return(True)
            if row['ORDER_DATE'][:4] == '2018':
                return(True)

In [161]:
internet_promo_users = [(row['CLIENT_NUMBER'], row['ORDER_NUMBER'], row['ORDER_DATE'][:7]) for i, row in tqdm(CMD.iterrows()) if desired_group(row)]

7323227it [06:46, 18036.61it/s]


In [162]:
len(internet_promo_users)

51407

In [163]:
len(set([pair[0] for pair in internet_promo_users])),len(set([pair[1] for pair in internet_promo_users]))

(41804, 51407)

In [164]:
net_catalog_order = [pair[1] for pair in internet_promo_users]

In [165]:
net_catalog_client = list(set([pair[0] for pair in internet_promo_users]))

In [166]:
len(net_catalog_client)

41804

In [ ]:
{identified_order_channels[i]:res_value_count[i] for i in range(len(identified_order_channels))}

We create a dico in order to be able to easily seperate the order ids and link them to the specific datasets

In [167]:
all_periods = list(set([tri[2] for tri in internet_promo_users]))

In [168]:
orders_by_date = {}
for tri in tqdm(internet_promo_users):
    if tri[2] not in orders_by_date.keys():
        orders_by_date[tri[2]] = []
    else:
        orders_by_date[tri[2]].append(tri[1])

100%|██████████| 51407/51407 [00:00<00:00, 1126405.73it/s]


In [169]:
client_by_date = {}
for tri in tqdm(internet_promo_users):
    if tri[2] not in client_by_date.keys():
        client_by_date[tri[2]] = []
    else:
        client_by_date[tri[2]].append(tri[0])

100%|██████████| 51407/51407 [00:00<00:00, 1107293.33it/s]


In [170]:
{k:len(v) for k,v in orders_by_date.items()}

{'2017-08': 5966,
 '2017-09': 4698,
 '2017-10': 5118,
 '2017-11': 5889,
 '2017-12': 4919,
 '2018-01': 6206,
 '2018-02': 3457,
 '2018-03': 5472,
 '2018-04': 4006,
 '2018-05': 4878,
 '2018-06': 787}

In [171]:
client_by_date

{'2017-08': [-6121063729151629387,
  4428260290674402911,
  -5058707774106742013,
  7280544278616931617,
  -4669298162092950526,
  1356833652490810573,
  432948164082238622,
  -940655217411426198,
  5871876916613627397,
  7106759981041398703,
  -2417163614231382136,
  -8974800123977524097,
  5600899696747637406,
  1960504506732313251,
  1679962373447437320,
  -4611559759304801986,
  -3494022014163969111,
  5017425694641111909,
  -671470348883263000,
  2235295082193521283,
  7951961390168433896,
  -5466927105228935665,
  3912878024541669794,
  -798852367487434152,
  -8343250787474410051,
  -8021092381084520664,
  -4851819729449491810,
  3739186217188391292,
  4599011098984479165,
  -7314884661943733362,
  -5090713327976249926,
  -8330990092460141911,
  -6936508765650177115,
  6381247444478916387,
  -1505906012453673676,
  -2990521614413960749,
  5872438721577605198,
  -6374456152464055858,
  -3999927607328122580,
  1434652420291857361,
  -1862289158792894540,
  8268245228438210144,
  -9

In [172]:
client_by_date = {date: list(set(clients)) for date, clients in client_by_date.items()}
{k:len(v) for k,v in client_by_date.items()}

{'2017-08': 5739,
 '2017-09': 4557,
 '2017-10': 4981,
 '2017-11': 5715,
 '2017-12': 4802,
 '2018-01': 6005,
 '2018-02': 3378,
 '2018-03': 5285,
 '2018-04': 3885,
 '2018-05': 4711,
 '2018-06': 775}

In [173]:
for key in all_periods:
    print(len(client_by_date[key]) <= len(orders_by_date[key]))

True
True
True
True
True
True
True
True
True
True
True


number of unique clients compared with number of unique orders is consistant, let's save this to a pickle for easier access later on

In [174]:
with open('data/client_by_date.pickle', 'wb') as handle:
    pickle.dump(client_by_date, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/orders_by_date.pickle', 'wb') as handle:
    pickle.dump(orders_by_date, handle, protocol=pickle.HIGHEST_PROTOCOL)

Loading in the pickles

In [175]:
with open('data/client_by_date.pickle', 'rb') as handle:
    client_by_date = pickle.load(handle)
    
with open('data/orders_by_date.pickle', 'rb') as handle:
    orders_by_date = pickle.load(handle)

Let's take a look at at least one month the clients behaviour to see if there is anything worth noting

We link the client numbers to the log files by the visitor ID variable in the conversion table

In [177]:
conver.head()

,Unnamed: 0,VISITOR_ID,CLIENT_NUMBER
0,0,153926,2117916268519517296
1,1,153987,-3983308646720649306
2,2,153992,-2525192206264180159
3,3,153996,1543724696132514059
4,4,154002,-3753347333864614269


In [176]:
client_by_date.keys()

dict_keys(['2017-08', '2017-09', '2017-11', '2017-10', '2017-12', '2018-01', '2018-02', '2018-03', '2018-05', '2018-04', '2018-06'])